In [ ]:
import sys
sys.path.insert(0, 'backend_functions')
from scipy.stats import norm
from matplotlib.colors import LogNorm
import xsec_functions

import importlib
import uproot
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd

importlib.reload(xsec_functions)
from xsec_functions import *

import selection_functions
import awkward

import ROOT

from ROOT import TH2D, TH1D, TH1F

import top 
from top import *

In [ ]:
importlib.reload(selection_functions)
from selection_functions import *

In [ ]:
fhc_data_pot = parameters(ISRUN3=False)['beamon_pot']
rhc_data_pot = parameters(ISRUN3=True)['beamon_pot']

In [ ]:
fold = "nuselection"
tree = "NeutrinoSelectionFilter"

fhc_nue = uproot.open(parameters(ISRUN3=False)['full_ntuple_path']+parameters(ISRUN3=False)['NUE']+".root")[fold][tree]
rhc_nue = uproot.open(parameters(ISRUN3=True)['full_ntuple_path']+parameters(ISRUN3=True)['NUE']+".root")[fold][tree]

uproot_nue = [fhc_nue, rhc_nue]
              

In [ ]:
variables = [
    "selected", "nu_pdg", 
    "trk_score_v", 
    "shr_tkfit_dedx_Y",
    "ccnc", 
    "n_tracks_contained", 
    "NeutrinoEnergy2",
    "reco_nu_vtx_sce_x","reco_nu_vtx_sce_y","reco_nu_vtx_sce_z",
    "true_nu_vtx_x", "true_nu_vtx_y" , "true_nu_vtx_z", 
    "nproton", "nu_e", "n_showers_contained",
    "shr_score", 
    "trk_energy", "tksh_distance", "tksh_angle",
    "npi0", 
    "shr_energy_tot_cali", "shr_energy_cali", 
    "nslice",
    "contained_fraction",
    "npion", "run", 
    "shrmoliereavg", 
    "shrsubclusters0","shrsubclusters1","shrsubclusters2", 
    "shr_tkfit_npointsvalid","shr_tkfit_npoints", "true_e_visible", "trkshrhitdist2", 
    "weightSplineTimesTune", "weightTune", "true_nu_px", "true_nu_py", "true_nu_pz", 
    "elec_e", "proton_e", "mc_px", "mc_py", "mc_pz", "elec_px", "elec_py", "elec_pz", 
    "swtrig_pre", "ppfx_cv", "mc_pdg", 'opening_angle']

In [ ]:
fhc_nue = fhc_nue.pandas.df(variables, flatten=False)

In [ ]:
rhc_nue = rhc_nue.pandas.df(variables, flatten=False)

In [ ]:
fhc_nue = addAngles(fhc_nue)
rhc_nue = addAngles(rhc_nue)

In [ ]:

df_v = [fhc_nue, rhc_nue]
    
for i,df in enumerate(df_v):
    up = uproot_nue[i]
    trk_llr_pid_v = up.array('trk_llr_pid_score_v')
    trk_id = up.array('trk_id')-1 # I think we need this -1 to get the right result
    trk_llr_pid_v_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_llr_pid_v,trk_id)])
    df['trkpid'] = trk_llr_pid_v_sel
    df['subcluster'] = df['shrsubclusters0'] + df['shrsubclusters1'] + df['shrsubclusters2']

    df['NeutrinoEnergy2_GeV'] = df['NeutrinoEnergy2']/1000
    df['is_signal'] = np.where((df.swtrig_pre == 1) 
                             & (df.nu_pdg==12) & (df.ccnc==0) & (df.nproton>0) & (df.npion==0) & (df.npi0==0)
                             & (10 <= df.true_nu_vtx_x) & (df.true_nu_vtx_x <= 246)
                             & (-106 <= df.true_nu_vtx_y) & (df.true_nu_vtx_y <= 106)
                             & (10 <= df.true_nu_vtx_z) & (df.true_nu_vtx_z <= 1026), True, False)
    
    

In [ ]:
# all we care about is signal 

fhc_signal = fhc_nue.query('swtrig_pre==1 and is_signal==True')
rhc_signal = rhc_nue.query('swtrig_pre==1 and is_signal==True')

df_v = [fhc_signal, rhc_signal]

In [ ]:
# cuts applied for bad weights 

for i,df in enumerate(df_v):

    df.loc[ df['weightSplineTimesTune'] <= 0, 'weightSplineTimesTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] == np.inf, 'weightSplineTimesTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] > 100, 'weightSplineTimesTune' ] = 1.
    df.loc[ np.isnan(df['weightSplineTimesTune']) == True, 'weightSplineTimesTune' ] = 1.

    df.loc[ df['weightTune'] <= 0, 'weightTune' ] = 1.
    df.loc[ df['weightTune'] == np.inf, 'weightTune' ] = 1.
    df.loc[ df['weightTune'] > 100, 'weightTune' ] = 1.
    df.loc[ np.isnan(df['weightTune']) == True, 'weightTune' ] = 1.  
    
    df['totweight_intrinsic'] = df['ppfx_cv']*df['weightSplineTimesTune']
    

In [ ]:
fhc_signal = pot_scale(fhc_signal, 'intrinsic', False)
rhc_signal = pot_scale(rhc_signal, 'intrinsic', True)

In [ ]:
fhc_signal['totweight_data'] = fhc_signal['ppfx_cv']*fhc_signal['weightSplineTimesTune']*fhc_signal['pot_scale']
rhc_signal['totweight_data'] = rhc_signal['ppfx_cv']*rhc_signal['weightSplineTimesTune']*rhc_signal['pot_scale']

### apply bdt model

In [ ]:
fhc_signal = fhc_signal.query(BDT_LOOSE_CUTS)
rhc_signal = rhc_signal.query(BDT_LOOSE_CUTS)

In [ ]:
print(parameters(ISRUN3=False)['bdt_score_cut'])
print(parameters(ISRUN3=True)['bdt_score_cut'])

In [ ]:
# FHC FIRST

bdt_model = xgb.Booster({'nthread': 4})
bdt_model.load_model(parameters(ISRUN3=False)['bdt_model'])

# clean datasets 
for column in training_parameters:
    fhc_signal.loc[(fhc_signal[column] < -1.0e37) | (fhc_signal[column] > 1.0e37), column] = np.nan
    
# create testing dmatrix 
fhc_signal_test = xgb.DMatrix(data=fhc_signal[training_parameters])
    
# apply the bdt selection
preds = bdt_model.predict(fhc_signal_test)

# add columns for plotting 
fhc_signal['BDT_score'] = preds

In [ ]:
np.sum(fhc_signal.query(BDT_LOOSE_CUTS+' and BDT_score>'+str(parameters(ISRUN3=False)['bdt_score_cut'])).totweight_data)

In [ ]:
# NOW DO RHC 

bdt_model = xgb.Booster({'nthread': 4})
bdt_model.load_model(parameters(ISRUN3=False)['bdt_model'])

# clean datasets 
for column in training_parameters:
    rhc_signal.loc[(rhc_signal[column] < -1.0e37) | (rhc_signal[column] > 1.0e37), column] = np.nan
    
# create testing dmatrix 
rhc_signal_test = xgb.DMatrix(data=rhc_signal[training_parameters])
    
# apply the bdt selection
preds = bdt_model.predict(rhc_signal_test)

# add columns for plotting 
rhc_signal['BDT_score'] = preds

sum(rhc_signal.query(BDT_LOOSE_CUTS+' and BDT_score>'+str(parameters(ISRUN3=True)['bdt_score_cut'])).totweight_data)

### Play with binning

In [ ]:
fhc_selected_signal = fhc_signal.query(BDT_LOOSE_CUTS+' and BDT_score>'+str(parameters(ISRUN3=False)['bdt_score_cut']))
rhc_selected_signal = rhc_signal.query(BDT_LOOSE_CUTS+' and BDT_score>'+str(parameters(ISRUN3=True)['bdt_score_cut']))

In [ ]:
importlib.reload(xsec_functions)
from xsec_functions import *

In [ ]:
xvar = 'tksh_angle'
true_var = 'opening_angle'

In [ ]:
## detector resolution 

# fhc 
true_reco_res(true_var, xvar, fhc_selected_signal, ISRUN3=False, ymax=None, save=False)

# rhc 
true_reco_res(true_var, xvar, rhc_selected_signal, ISRUN3=True, ymax=None, save=False)


In [ ]:
#shower energy
#[0.07, 0.26, 0.45, 0.64, 0.83, 1.02, 1.4, 4.60] # update to even out the last bin (go as far as truth level?)

# visible energy 
#bins = [0.09, 0.36, 0.56, 0.76, 0.96, 1.16, 1.36, 1.65, 2.25, 6.5]

# tksh_angle



In [ ]:
# min & max reco/truth values 

bins = [-1, np.cos(3*np.pi/4), np.cos(5*np.pi/8), 0, np.cos(3*np.pi/8), np.cos(np.pi/4), 1]

fig = plt.figure(figsize=(8, 5))



x = plt.hist([fhc_selected_signal[xvar], rhc_selected_signal[xvar]], bins, color=['orange', 'peru'], 
         weights=[fhc_selected_signal['totweight_data'], rhc_selected_signal['totweight_data']], 
                  label=['FHC ($2\\times10^{20}$ POT)', 'RHC ($5\\times10^{20}$ POT)'], stacked=True)

plt.legend(fontsize=14, loc='upper left')
#plt.title("FHC RUN 1 ("+str(fhc_data_pot)+' POT)', fontsize=15)

plt.xlabel("$cos$ $\\theta_{ep}$", fontsize=15)
plt.ylabel("Signal event rate", fontsize=15)

plt.xticks(bins)
plt.ylim(0, 50)

plt.show()

In [ ]:
[-1, -0.5, np.cos(95*np.pi/180), np.cos(85*np.pi/180), 0.5, 1]

In [ ]:
# min & max reco/truth values 

bins = [-1, -0.5, np.cos(95*np.pi/180), np.cos(85*np.pi/180), 0.5, 1]

fig = plt.figure(figsize=(8, 5))



x = plt.hist([fhc_selected_signal[xvar], rhc_selected_signal[xvar]], bins, color=['orange', 'peru'], 
         weights=[fhc_selected_signal['totweight_data'], rhc_selected_signal['totweight_data']], 
                  label=['FHC ($2\\times10^{20}$ POT)', 'RHC ($5\\times10^{20}$ POT)'], stacked=True)

plt.legend(fontsize=14, loc='upper left')
#plt.title("FHC RUN 1 ("+str(fhc_data_pot)+' POT)', fontsize=15)

plt.xlabel("$cos$ $\\theta_{ep}$", fontsize=15)
plt.ylabel("Signal event rate", fontsize=15)

plt.xticks(bins)
plt.ylim(0, 50)

plt.show()

In [ ]:
# min & max reco/truth values 

#bins = [-1, -0.7, -0.42, -0.14, 0.14, 0.42, 0.7, 1]
bins = [-1, -0.5, np.cos(100 * np.pi/180), np.cos(80 * np.pi/180), 0.5, 1]

fig = plt.figure(figsize=(8, 5))



x = plt.hist([fhc_selected_signal[xvar], rhc_selected_signal[xvar]], bins, color=['orange', 'peru'], 
         weights=[fhc_selected_signal['totweight_data'], rhc_selected_signal['totweight_data']], 
                  label=['FHC ($2\\times10^{20}$ POT)', 'RHC ($5\\times10^{20}$ POT)'], stacked=True)

plt.legend(fontsize=14, loc='upper left')
#plt.title("FHC RUN 1 ("+str(fhc_data_pot)+' POT)', fontsize=15)

plt.xlabel("$cos$ $\\theta_{ep}$", fontsize=15)
plt.ylabel("Signal event rate", fontsize=15)

plt.xticks(bins)
plt.ylim(0, 50)

plt.show()



In [ ]:
bins

In [ ]:
xvar='NeutrinoEnergy2_GeV'
#bins = [0.05, 0.45, 0.65, 0.85, 1.05, 1.25, 1.65, 2.25, 7]
bins = [0.05, 0.45, 0.65, 0.85, 1.05, 1.35, 1.85, 7]

fig = plt.figure(figsize=(8, 5))

fhc_reco = plt.hist(fhc_selected_signal[xvar], bins, color='orange', 
         weights=fhc_selected_signal['totweight_data'], label='reco FHC')[0]

#fhc_true = plt.hist(fhc_selected_signal[true_var], bins, 
#         weights=fhc_selected_signal['totweight_data'], 
#         color='saddlebrown', label='true FHC', histtype='step', linewidth=2, linestyle='--')[0]

plt.legend(fontsize=14)


#plt.xlabel("$cos$ $\\theta_{ep}$", fontsize=15)
plt.ylabel("Signal event rate", fontsize=15)
#plt.xticks(bins)

plt.show()

fig = plt.figure(figsize=(8, 5))

rhc_reco = plt.hist(rhc_selected_signal[xvar], bins, color='orange', 
         weights=rhc_selected_signal['totweight_data'], label='reco RHC')[0]

#rhc_true = plt.hist(rhc_selected_signal[true_var], bins, 
#         weights=rhc_selected_signal['totweight_data'], 
#         color='saddlebrown', label='true RHC', histtype='step', linewidth=2, linestyle='--')[0]

plt.legend(fontsize=14)

#plt.xlabel("$cos$ $\\theta_{ep}$", fontsize=15)
plt.ylabel("Signal event rate", fontsize=15)
#plt.xticks(bins)


plt.show()

fhc_reco+rhc_reco

In [ ]:
rhc_reco



In [ ]:
## smearing matrix 
fig = plt.figure(figsize=(11, 8))

hout = plt.hist2d(fhc_selected_signal[true_var], fhc_selected_signal[xvar], [0.07, 0.26, 0.45, 0.64, 0.83, 1.02, 1.4, 4.5], 
                              weights=fhc_selected_signal.totweight_data, cmap='OrRd', cmin=0.01)

smear_array = hout[0].T

for i in range(len(bins)-1): # reco bins i (y axis)
        for j in range(len(bins)-1): # true bins j (x axis)
            if smear_array[i][j]>7: 
                col='white'
            else: 
                col='black'

            binx_centers = hout[1][j]+(hout[1][j+1]-hout[1][j])/2
            biny_centers = hout[2][i]+(hout[2][i+1]-hout[2][i])/2

            if not np.isnan(smear_array[i][j]):
                plt.text(binx_centers, biny_centers, round(smear_array[i][j], 1), 
                            color=col, ha="center", va="center", fontsize=12)
                
                
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

cbar = plt.colorbar()
#cbar.set_label('$\\nu$ / '+pot+' POT', fontsize=15)

plt.xlabel('True Electron Energy [GeV]', fontsize=15)
plt.ylabel('Reco Electron Energy [GeV]', fontsize=15)

#plt.title(title+' - Smearing '+str(parameters(isrun3)['beamon_pot'])+' POT', fontsize=15)

cbar.ax.tick_params(labelsize=14)
plt.show()


In [ ]:
print('min reco', min(fhc_selected_signal[xvar]))
print('max reco', max(fhc_selected_signal[xvar]))

print('min true', min(fhc_selected_signal[true_var]))
print('max true', max(fhc_selected_signal[true_var]))

In [ ]:

print('min reco', min(rhc_selected_signal[xvar]))
print('max reco', max(rhc_selected_signal[xvar]))

print('min true', min(rhc_selected_signal[true_var]))
print('max true', max(rhc_selected_signal[true_var]))
